In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

import pandas as pd
import numpy as np

from utils.preprocessing import preprocess_df
from utils.df_loader import load_adult_df, load_compas_df, load_german_df, load_diabetes_df, load_breast_cancer_df
from utils.evaluation import get_evaluations, EvaluationMatrix

from utils.load import load_result_from_csv, load_datapoints_from_npy
from utils.models import load_models
from utils.models import save_model_performance

seed = 42

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None # suppress "SettingWithCopyWarning" warning

In [3]:
datasets = [
        # "adult",
        # "german",
        # "compas",
        # "diabetes",
        "breast_cancer",
        ]

models = ["dt","gbc","lr","svc","nn_2"] # "dt","gbc","lr","svc",


attack_list = [
        'deepfool', 
        'carlini_l_2', 
        'carlini_l_inf', 
        'lowprofool_l_2', 
        'lowprofool_l_inf', 
        'fgsm_l_1',
        'fgsm_l_2',
        'fgsm_l_inf',
        'bim',
        # 'boundary', 
        # 'hopskipjump_l_2', 'hopskipjump_l_inf'
        ]


In [4]:
def get_loading_fn(dataset_name):
    if dataset_name == 'adult':
        dataset_loading_fn = load_adult_df
    elif dataset_name == 'german':
        dataset_loading_fn = load_german_df
    elif dataset_name == 'compas':
        dataset_loading_fn = load_compas_df
    elif dataset_name == 'diabetes':
        dataset_loading_fn = load_diabetes_df
    elif dataset_name == 'breast_cancer':
        dataset_loading_fn = load_breast_cancer_df
    else:
        raise Exception("Unsupported dataset")
    return dataset_loading_fn


## Combine dataframe

In [5]:
# for dataset_name in datasets:
#     for attack in attack_list:
#         folder_name = f"{attack}_{dataset_name}"

#         ## check if the folder exist

#         if os.path.isdir(f'./results/{folder_name}'):
#             for model_name in models:

#                 dfs = []
#                 file_name = f'{folder_name}_{model_name}_result.csv'
#                 destination_path = f'./results/{folder_name}/{file_name}'

#                 if os.path.isfile(f'./results/{folder_name}/{folder_name}_{model_name}_result_1.csv'):
#                     for i in range(0,10):
#                         dataset_path = (
#                             f"{attack}_{dataset_name}_{model_name}_result_{i}.csv"
#                         )
#                         dfs.append(pd.read_csv(f"./results/{folder_name}/{dataset_path}"))

#                     ### Combine dfs
#                     complete_df = pd.DataFrame([], columns=dfs[0].columns)
#                     for l in range(len(dfs[0])):
#                         for df in dfs:
#                             complete_df = complete_df.append(df.iloc[l : l + 1])

#                     ### Save dfs
#                     complete_df.to_csv(destination_path)
#                     print(f"Have saved combined sheet to {destination_path}")

## Check wheather white-box attack output same results

In [77]:
# def are_ndarrays_same(ndarrays):
#   # Create an empty matrix of size len(ndarrays) x len(ndarrays)
#   results_matrix = np.empty((len(ndarrays), len(ndarrays)))

#   # Iterate through each pair of ndarrays and check if they are the same
#   for i in range(len(ndarrays)):
#     for j in range(len(ndarrays)):
#       if i == j:
#         # If the indices are the same, mark it as True in the results matrix
#         results_matrix[i][j] = True
#       else:
#         # Compare the ndarrays using the numpy.array_equal function
#         results_matrix[i][j] = np.array_equal(ndarrays[i], ndarrays[j])
#   return results_matrix

In [78]:
# for dataset_name in datasets:
#     for attack in attack_list:
#         for model_name in models:
#             ndarrays = []
#             for running_times in range(0,10):
#                 ndarrays.append(load_datapoints_from_npy(attack, dataset_name, model_name, running_times, adv=True))
#             print(f'{dataset_name} - {attack} - {model_name}')
#             print(are_ndarrays_same(ndarrays).min())




## Evaluation

In [6]:
def get_dic_from_metric(all_metric):

    dataset_arr = []
    attack_arr = []
    model_arr = []
    metric_arr = []
    value_arr = []

    for dataset, dic1 in all_metric.items():
        for attack, dic2 in dic1.items():
            for model, dic3 in dic2.items():
                for metric, value in dic3.items():
                    dataset_arr.append(dataset)
                    attack_arr.append(attack)
                    model_arr.append(model)
                    metric_arr.append(metric)
                    value_arr.append(value)

    table = {
            'Dataset': dataset_arr,
            'Attack': attack_arr,
            'Model': model_arr,
            'Metric': metric_arr,
            'Value': value_arr,
        }

    return table




In [81]:
#### Select dataset ####

all_metric = {}

for dataset_name in datasets:
    all_metric[dataset_name]={}

    df_info = preprocess_df(get_loading_fn(dataset_name))
    for attack in attack_list:
        all_metric[dataset_name][attack]={}

        folder_name = f'{attack}_{dataset_name}'
        for model_name in models:

            file_name = f'{folder_name}_{model_name}_result_0.csv'
            result_path = f'./results/{folder_name}/{file_name}'
            if os.path.isfile(result_path):
                result_df = pd.read_csv(result_path)
                evaluation_df, metric = get_evaluations(result_df=result_df, 
                    df_info=df_info, 
                    matrix = [
                        EvaluationMatrix.L1, 
                        EvaluationMatrix.L2, 
                        EvaluationMatrix.Linf,
                        EvaluationMatrix.MAD, 
                        EvaluationMatrix.Mahalanobis,
                        EvaluationMatrix.Sparsity, 
                        EvaluationMatrix.Perturbation_Sensitivity,
                        EvaluationMatrix.Neighbour_Distance,
                        ])
                
                all_metric[dataset_name][attack][model_name] = metric

                csv_save_result_path = f'results/{folder_name}/eval_{file_name}'
                evaluation_df.to_csv(csv_save_result_path)
                print(f"Have saved file to {csv_save_result_path}")


Have saved file to results/deepfool_breast_cancer/eval_deepfool_breast_cancer_lr_result_0.csv
Have saved file to results/deepfool_breast_cancer/eval_deepfool_breast_cancer_svc_result_0.csv
Have saved file to results/deepfool_breast_cancer/eval_deepfool_breast_cancer_nn_2_result_0.csv
Have saved file to results/carlini_l_2_breast_cancer/eval_carlini_l_2_breast_cancer_lr_result_0.csv
Have saved file to results/carlini_l_2_breast_cancer/eval_carlini_l_2_breast_cancer_svc_result_0.csv
Have saved file to results/carlini_l_2_breast_cancer/eval_carlini_l_2_breast_cancer_nn_2_result_0.csv
Have saved file to results/carlini_l_inf_breast_cancer/eval_carlini_l_inf_breast_cancer_lr_result_0.csv
Have saved file to results/carlini_l_inf_breast_cancer/eval_carlini_l_inf_breast_cancer_svc_result_0.csv
Have saved file to results/carlini_l_inf_breast_cancer/eval_carlini_l_inf_breast_cancer_nn_2_result_0.csv
Have saved file to results/lowprofool_l_2_breast_cancer/eval_lowprofool_l_2_breast_cancer_lr_resu

In [82]:
im_results = pd.DataFrame.from_dict(get_dic_from_metric(all_metric))

In [83]:
im_results.to_csv(f"./results/{datasets[0]}_evaluation_results.csv",index=False)
# im_results.to_csv(f"./results/diabetes_evaluation_results.csv",index=False)
# im_results.to_csv(f"./results/carlini_l2_adult_table.csv",index=False)

In [84]:
im_results

,Dataset,Attack,Model,Metric,Value
0,breast_cancer,deepfool,lr,eval_L1,8.025208
1,breast_cancer,deepfool,lr,eval_L2,1.700544
2,breast_cancer,deepfool,lr,eval_Linf,0.523994
3,breast_cancer,deepfool,lr,eval_MAD,2.507095
4,breast_cancer,deepfool,lr,eval_Mahalanobis,0.842675
...,...,...,...,...,...
115,breast_cancer,lowprofool_l_inf,nn_2,eval_MAD,2.547293
116,breast_cancer,lowprofool_l_inf,nn_2,eval_Mahalanobis,0.454430
117,breast_cancer,lowprofool_l_inf,nn_2,eval_Sparsity,29.937500
118,breast_cancer,lowprofool_l_inf,nn_2,eval_Perturbation_Sensitivity,3.270870


## Model Performance

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


def compare_ndarrays(arr1, arr2):
    if arr1.shape != arr2.shape:
        raise ValueError("Input arrays have different shapes")
    return np.where(arr1 == arr2, 0, 1)

def get_attack_success_accuracy(models, model, input_array, adv_array, groundtruth):

    if model == 'dt':
        predictions = models['dt'].predict(input_array)
        adv_predictions = models['dt'].predict(adv_array)
    if model == 'rfc':
        predictions = models['rfc'].predict(input_array)
        adv_predictions = models['rfc'].predict(adv_array)
    if model == 'svc':
        predictions = models['svc'].predict(input_array)
        adv_predictions = models['svc'].predict(adv_array)
    if model == 'lr':
        predictions = models['lr'].predict(input_array)
        adv_predictions = models['lr'].predict(adv_array)
    if model == 'gbc':
        predictions = models['gbc'].predict(input_array)
        adv_predictions = models['gbc'].predict(adv_array)
    if model == 'nn':
        predictions = (models['nn'].predict(input_array) > 0.5).flatten().astype(int)
        adv_predictions = (models['nn'].predict(adv_array) > 0.5).flatten().astype(int)
    if model == 'nn_2':
        predictions = models['nn_2'].predict(input_array).argmax(axis=1).flatten().astype(int)
        adv_predictions = models['nn_2'].predict(adv_array).argmax(axis=1).flatten().astype(int)


    pred_attack_success = compare_ndarrays(predictions, adv_predictions).mean()
    groundtruth_attack_success = compare_ndarrays(groundtruth, adv_predictions).mean()
    original_accuracy = accuracy_score(groundtruth, predictions)
    robust_accuracy = accuracy_score(groundtruth, adv_predictions)



    dict = {
        'pred_attack_success':pred_attack_success, 
            'original_accuracy': original_accuracy, 
            'robust_accuracy': robust_accuracy}

    print(dict)
    return dict



In [8]:
from utils.models import save_model_performance

In [9]:
#### Select dataset ####

all_performance = {}

for dataset_name in datasets:
    all_performance[dataset_name]={}

    df_info = preprocess_df(get_loading_fn(dataset_name))
    train_df, test_df = train_test_split(
        df_info.dummy_df, train_size=0.8, random_state=seed, shuffle=True
    )
    X_train = np.array(train_df[df_info.ohe_feature_names])
    y_train = np.array(train_df[df_info.target_name])
    X_test = np.array(test_df[df_info.ohe_feature_names])
    y_test = np.array(test_df[df_info.target_name])

    X_test_num = len(X_test) - (len(X_test)%64)
    X_test_re=X_test[0:X_test_num]
    y_test_num = len(y_test) - (len(y_test)%64)
    y_test_re=y_test[0:y_test_num]

    predict_model = load_models(X_train.shape[-1], dataset_name)

    for attack in attack_list:
        all_performance[dataset_name][attack]={}

        folder_name = f'{attack}_{dataset_name}'
        for model_name in models:

            adv_arr_name = f'{folder_name}_{model_name}_arr_adv_0.npy'
            adv_arr_path = f'./datapoints/{folder_name}/{adv_arr_name}'
            
            if os.path.isfile(adv_arr_path):
                adv_arr = load_datapoints_from_npy(attack, dataset_name, model_name, 0, adv=True)

                all_performance[dataset_name][attack][model_name] = get_attack_success_accuracy(predict_model, model_name, X_test_re, adv_arr, y_test_re)

                # m = save_model_performance(predict_model,dataset_name, adv_arr, y_test_re)



1/1 [==============================] - 0s 48ms/step
{'pred_attack_success': 1.0, 'original_accuracy': 0.984375, 'robust_accuracy': 0.015625}
{'pred_attack_success': 1.0, 'original_accuracy': 0.984375, 'robust_accuracy': 0.015625}
2/2 [==============================] - 0s 2ms/step
{'pred_attack_success': 1.0, 'original_accuracy': 0.96875, 'robust_accuracy': 0.03125}
{'pred_attack_success': 0.921875, 'original_accuracy': 0.984375, 'robust_accuracy': 0.09375}
{'pred_attack_success': 0.03125, 'original_accuracy': 0.984375, 'robust_accuracy': 0.953125}
2/2 [==============================] - 0s 2ms/step
{'pred_attack_success': 0.859375, 'original_accuracy': 0.96875, 'robust_accuracy': 0.171875}
{'pred_attack_success': 0.125, 'original_accuracy': 0.984375, 'robust_accuracy': 0.890625}
{'pred_attack_success': 0.140625, 'original_accuracy': 0.984375, 'robust_accuracy': 0.875}
2/2 [==============================] - 0s 2ms/step
{'pred_attack_success': 0.234375, 'original_accuracy': 0.96875, 'robu

In [10]:

performance_df = pd.DataFrame.from_dict(get_dic_from_metric(all_performance))


In [13]:
performance_df.to_csv(f"./results/{datasets[0]}_performance_results.csv",index=False)

In [12]:
performance_df

,Dataset,Attack,Model,Metric,Value
0,breast_cancer,deepfool,lr,pred_attack_success,1.000000
1,breast_cancer,deepfool,lr,original_accuracy,0.984375
2,breast_cancer,deepfool,lr,robust_accuracy,0.015625
3,breast_cancer,deepfool,svc,pred_attack_success,1.000000
4,breast_cancer,deepfool,svc,original_accuracy,0.984375
...,...,...,...,...,...
76,breast_cancer,bim,svc,original_accuracy,0.984375
77,breast_cancer,bim,svc,robust_accuracy,0.015625
78,breast_cancer,bim,nn_2,pred_attack_success,1.000000
79,breast_cancer,bim,nn_2,original_accuracy,0.968750
